In [44]:
import os, math, subprocess
import datetime as dt

#data wrangling
import numpy as np
import pandas as pd

#visualization
import seaborn as sns
import matplotlib.pyplot as plt

#excel library
import xlrd

#importing Statsmodels
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse
from scipy.stats import chisquare
#Scikitlearn libraries
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE, chi2
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from yellowbrick.regressor import ResidualsPlot, PredictionError, AlphaSelection
#decision tree
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn import tree
#decision tree regressor visualize
from sklearn.externals.six import StringIO
from IPython.display import Image
import pydotplus


In [45]:
#Loading data
salary = pd.read_excel('salary features.xlsx')
salary.describe()

,payment_count,payment_sum,avg_income_past_3_months,annual_salary_after_tax,weekly_income,weekly_tax_withholding,annual_salary
count,100.000000,100.000000,100.00000,100.0000,100.000000,100.000000,100.000000
mean,8.830000,16765.768500,2297.27390,61704.7700,1188.300000,376.298976,80902.360000
std,3.651636,6741.543716,1397.78397,29016.8551,557.617473,199.761215,39932.764812
min,2.000000,7155.960000,576.00000,2385.0000,46.990000,8.738100,2420.000000
25%,6.000000,11591.332500,1153.31000,45495.5000,879.990000,261.724923,59246.250000
50%,7.000000,14813.485000,1949.43000,57435.5000,1105.490000,339.662950,75131.500000
75%,13.000000,21332.515000,2886.72500,80170.0000,1542.990000,497.900400,106071.000000
max,14.000000,35343.920000,8835.98000,134577.0000,2588.990000,905.840400,181731.000000


In [46]:
#Choose variables for input:
predict = salary
predict = predict.drop('customer_id',1)
predict = predict.drop('first_name',1)
predict = predict.drop('txn_description',1)
predict = predict.drop('payment_type',1)
predict.head()
#Features are 'date','payment_count','payment_sum','avg_income_past_3_months', 'annual_salary_after_tax', 'weekly_income', 'weekly_tax_withholding'

,payment_count,payment_sum,avg_income_past_3_months,annual_salary_after_tax,weekly_income,weekly_tax_withholding,annual_salary
0,13,33644.13,2588.01,134577,2588.99,905.8404,181731
1,13,33081.88,2544.76,132328,2545.99,889.0704,178623
2,13,31762.12,2443.24,127048,2443.99,849.2904,171251
3,13,29644.68,2280.36,118579,2280.99,785.7204,159469
4,7,30837.10,4405.30,114538,2203.99,755.6904,153834


In [47]:
X = predict.drop('annual_salary',1)   #Feature Matrix
y = predict['annual_salary']          #Target Variable
# create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 100)
print ('X training set:', X_train.shape,'Y training set:', y_train.shape)
print ('X testing set:', X_test.shape, 'Y testing set:', y_test.shape)
#Top: 50% training set of table contains 7 columns
#Bottom: 50% testing set of table contains 7 columns

X training set: (80, 6) Y training set: (80,)
X testing set: (20, 6) Y testing set: (20,)


In [48]:
# Function to perform with Entropy
clf= DecisionTreeRegressor(max_depth=4)

# Train Decision Tree Classifer
clf.fit(X_train,y_train)

DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [49]:
#Predict the response for test dataset
y_pred = clf.predict(X_test)
y_pred

array([ 86475.        ,  63563.78571429,  96326.5       ,  86475.        ,
        86475.        , 107427.        ,  86475.        ,  96326.5       ,
        50882.16666667, 113549.8       ,  50882.16666667,  74320.9375    ,
        22514.25      ,  63563.78571429,   3069.        ,  50882.16666667,
        63563.78571429, 138977.6       ,  86475.        , 138977.6       ])

In [50]:
y_test = y_test.values
y_test

array([ 86863,  67210,  98974,  84235,  90162, 105619,  87814,  91843,
        56644, 112198,  47056,  74833,  19994,  62309,   3721,  55205,
        60280, 146986,  84767, 135400], dtype=int64)

In [51]:
#https://blog.quantinsti.com/decision-tree/
#https://www.datasciencecentral.com/profiles/blogs/price-optimisation-using-decision-tree-regression-tree
df_cc = pd.DataFrame({'y_pred': y_pred, 'y_test': list(y_test)}, columns=['y_pred', 'y_test'])
df_cc.describe()

,y_pred,y_test
count,20.000000,20.000000
mean,78360.102232,78605.650000
std,34130.346188,34427.451114
min,3069.000000,3721.000000
25%,60393.380952,59371.000000
50%,86475.000000,84501.000000
75%,96326.500000,93625.750000
max,138977.600000,146986.000000


In [68]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
#Annual Salary mean = 80902.36, our Mean Absolute Error = 2908.83 => 3.6% error => model accuracy = 96.4%
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 2850.9365773809527
Mean Squared Error: 11591499.764993783
Root Mean Squared Error: 3404.6291670303513


In [72]:
dtm = DecisionTreeRegressor(max_depth=4)

dtm.fit(X_train,y_train)
print("R-Squared on train dataset={}".format(dtm.score(X_train,y_train)))

dtm.fit(X_test,y_test)   
print("R-Squaredon test dataset={}".format(dtm.score(X_test,y_test)))

R-Squared on train dataset=0.9945818640585586
R-Squaredon test dataset=0.9969253995661307
